In [ ]:
def is_valid(controller):
    v0_l, v0_r = controller[0], controller[1]
    v1_l, v1_r = controller[2], controller[3]
    v0_l, v0_r, v1_l, v1_r = round(v0_l, 1), round(v0_r, 1), round(v1_l, 1), round(v1_r, 1)

    k = 0.5
    max_elem_score = max(-min(controller), max(controller))
    max_elem_score = -max_elem_score if max_elem_score < k else max_elem_score

    k_2 = 0.75
    magnitude_score = np.linalg.norm(controller)
    magnitude_score = -magnitude_score if magnitude_score < k_2 else magnitude_score

    k_3 = 0.3
    average_score = np.average(np.sqrt(np.power(controller, 2)))
    average_score = -average_score if average_score < k_3 else average_score

    # Sensor off magnitude (trial i)
    on_magnitude = (v0_l ** 2) + (v0_r ** 2)

    # Sensor on magnitude (trial i)
    off_magnitude = (v1_l ** 2) + (v1_r ** 2)

    # Spinning Detection (sensor off - trial ii)
    if v0_l == 0.0 and v0_r == 0.0:
        off_spin_variance = 1
    else:
        denom = v0_l if v0_l != 0.0 else v0_r
        off_spin_variance = min(abs((v0_l + v0_r) / denom), 1.0)

    # Spinning Detection (sensor on - trial ii)
    if v1_l == 0.0 and v1_r == 0.0:
        on_spin_variance = 0.0
    else:
        denom = v1_l if v1_l != 0.0 else v1_r
        on_spin_variance = min(abs((v1_l + v1_r) / denom), 1)

    # Mirror Property
    mirrored_controller = np.array([v0_l, v0_r, -v0_l, -v0_r])
    mirror_score = np.linalg.norm(mirrored_controller - np.array([v0_l, v0_r, v1_l, v1_r]))
    k_m = 0.3
    mirror_score = -5 if mirror_score < k_m else mirror_score

    # Independence Property
    independent_controller = np.array([v0_l, v0_r, v0_l, v0_r])
    indep = np.linalg.norm(independent_controller - np.array([v0_l, v0_r, v1_l, v1_r]))

    attributes = [
        indep,
        mirror_score,
        on_spin_variance,
        off_spin_variance,
        on_magnitude,
        off_magnitude,
        max_elem_score,
        magnitude_score,
        average_score,
    ]

    # weights = [5.3943, 4.5802, 3.3803, 1.7969, -4.1899, -3.9899, -7.3916, 2.5855, 10.1178]
    weights = [0.8993,  0.7878, 1.7404,  0.7404,  0.4437,  0.3982, -0.1233,  0.0905, 1.1693]
    mx = np.dot(np.array(attributes), np.array(weights))
    # return mx > 10.5
    return mx > 3.8

In [ ]:
import pygame
import torch
import time
import os
import sys
import numpy as np
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from NovelSwarmBehavior.novel_swarms.config.EvolutionaryConfig import GeneticEvolutionConfig
from NovelSwarmBehavior.novel_swarms.config.WorldConfig import RectangularWorldConfig
from NovelSwarmBehavior.novel_swarms.config.AgentConfig import DiffDriveAgentConfig
from NovelSwarmBehavior.novel_swarms.sensors.SensorSet import SensorSet
from NovelSwarmBehavior.novel_swarms.sensors.BinaryLOSSensor import BinaryLOSSensor
from NovelSwarmBehavior.novel_swarms.sensors.GenomeDependentSensor import GenomeBinarySensor
from NovelSwarmBehavior.novel_swarms.novelty.GeneRule import GeneBuilder
from NovelSwarmBehavior.novel_swarms.config.defaults import ConfigurationDefaults
from NovelSwarmBehavior.novel_swarms.novelty.GeneRule import GeneRule
from NovelSwarmBehavior.novel_swarms.config.OutputTensorConfig import OutputTensorConfig
from generation.evolution import ModifiedNoveltyArchieve
from PIL import Image
from sklearn_extra.cluster import KMedoids
import cv2
import numpy as np
import torch
from data.swarmset import ContinuingDataset, SwarmDataset
from networks.embedding import NoveltyEmbedding
from generation.evolution import ModifiedHaltingEvolution
from networks.archive import DataAggregationArchive
from hil.HIL import HIL
import time
from networks.ensemble import Ensemble

# dataset = ContinuingDataset("/home/connor/Desktop/Experiments/GECCO_2023/Brown-two-sensor-no-h/Experiment_1/1675638750-brown-two-sensor-no-h", folder_name="data", create=False)
dataset = ContinuingDataset("/home/connor/Desktop/Experiments/GECCO_2023/LL-two-sensor-no-heuristics/Experiment_2/1675267452-two-sensor", folder_name="data", create=False)
member = 0
network = "../checkpoints/ensembles/01-28-23-2S-HIL-A"
# network = None
if network is not None:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    ensemble = Ensemble(size=3, output_size=5, lr=0.0, learning_decay=0.0, decay_step=0, threshold=0.0, weight_decay=0.0, new_model=True)
    ensemble.load_ensemble(network, full=True)
    network = ensemble.ensemble[member]
    network.eval()

medoids = 20
cap_data = 10000
FILTERING = False
COMBINE = True

def resizeInput(X, w=200):
    frame = X.astype(np.uint8)
    resized = cv2.resize(frame, dsize=(w, w), interpolation=cv2.INTER_AREA)
    return resized

archive = ModifiedNoveltyArchieve()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for i in range(cap_data):
    anchor_encoding, genome, behavior = dataset[i][0], dataset[i][1], dataset[i][2]
    if network is not None:
        anchor_encoding = resizeInput(anchor_encoding, 50)
        anchor_encoding = torch.from_numpy(anchor_encoding).to(device).float()
        network.eval()
        embedding = network(anchor_encoding.unsqueeze(0)).squeeze(0).cpu().detach().numpy()
        if COMBINE:
            embedding = np.concatenate((behavior, embedding))
        archive.addToArchive(vec=embedding, genome=genome)
    else:
        if FILTERING and not is_valid(genome):
            continue
        archive.addToArchive(vec=behavior, genome=genome)

print(f"After filtering... {len(archive.archive)}")
kmedoids = KMedoids(n_clusters=medoids, random_state=0).fit(archive.archive)
medoids = kmedoids.medoid_indices_
labels = kmedoids.labels_

import matplotlib.pyplot as plt
for i, medoid_i in enumerate(medoids):
    medoid_image = dataset[medoid_i][0]
    print(dataset[medoid_i][1])
    plt.imshow(medoid_image, cmap="Greys")
    plt.show()

In [ ]:
is_valid([-0.4, 0.8, 0.9, -0.2, 0.6, 1.0, 0.6, -0.0, np.pi/6])